Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal. For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

We generate a 3D geometry and mesh using the OCC constructive solid geometry (CSG) modeler:

In [ ]:
from netgen.occ import *
cube = Box((0,0,0),(1,1,1))
cyl = Cylinder((0,0.5,0.5),X, r=0.2, h=1)
cube.faces.name = "outer"
cyl.faces.name = "cyl"
shape = cube-cyl
ea = { "euler_angles" : [-180, -80, 165] }
Draw(shape, **ea);

In [ ]:
ngmesh = OCCGeometry(shape).GenerateMesh(maxh=0.1)

mesh = Mesh(ngmesh).Curve(3)

for l in range(0):
    mesh.Refine()

Draw (mesh, **ea);

In [ ]:
fes = H1(mesh, order=3, dirichlet="outer", wb_withedges=False)
print ("we have", fes.ndof, "unknowns")

u,v = fes.TnT()

a = BilinearForm(grad(u)*grad(v)*dx)
f = LinearForm(1*v*dx)

# pre = preconditioners.MultiGrid(a)
pre = preconditioners.Local(a)
# pre = preconditioners.BDDC(a)

gfu = GridFunction(fes)

assemble system and setup preconditioner in parallel:

In [ ]:
with TaskManager():
    a.Assemble()
    f.Assemble()

solve the system using the preconditioned conjugate gradient method:

In [ ]:
from ngsolve.krylovspace import CGSolver
from time import time

ts = time()
with TaskManager():
    # inv = a.mat.Inverse(inverse="sparsecholesky", freedofs=fes.FreeDofs())
    inv = CGSolver(mat=a.mat, pre=pre, printrates='\r', maxiter=1000)
    gfu.vec.data = inv * f.vec
te = time()
print ("needed", te-ts, "seconds for", fes.ndof, "unknowns")

In [ ]:
Draw (gfu, **ea);